# Criptoanálise contra Criptografia Simétrica

## Algoritmo de Grover vs AES

In [1]:
from qat.lang import qrout
from qat.lang.AQASM import Program, QRoutine, CNOT, SWAP, H, X, CCNOT
from qat.qpus import get_default_qpu, PyLinalg
import numpy as np

# Algoritmo de Grover vs ChaCha

## Algoritmo ChaCha

In [2]:
@qrout
def add(a, b):
    n = len(a)
    for i in range(1, n):
        CNOT(b[i], a[i])
    for i in range(n-2, 0, -1):
        CNOT(b[i], b[i+1])
    for i in range(n-1):
        CCNOT(a[i], b[i], b[i+1])
    for i in range(n-1, 0, -1):
        CNOT(b[i], a[i])
        CNOT.ctrl()(a[i-1], b[i-1], b[i])
    for i in range(1, n-1):
        CNOT(b[i], b[i+1])
    for i in range(n):
        CNOT(b[i], a[i])


add([0,1,2], [3,4,5]).display()

In [3]:
@qrout
def quarterround(a,b):
    # a e b possuem tamanho 3 qubits
    n=3
    add(a,b)
    for i in range(1, n):
        CNOT(b[i], a[i])
    for i in range(n-2, 0, -1):
        CNOT(b[i], b[i+1])
    for i in range(n-1):
        CCNOT(a[i], b[i], b[i+1])
    for i in range(n-1, 0, -1):
        CNOT(b[i], a[i])
        CNOT.ctrl()(a[i-1], b[i-1], b[i])
    for i in range(1, n-1):
        CNOT(b[i], b[i+1])
    for i in range(n):
        CNOT(b[i], a[i])
    
    for i in range(n):
        CNOT(a[i], b[i])

    #RL(2, b)
    SWAP(b[2], b[1])
    SWAP(b[1], b[0])
    SWAP(b[2], b[1])
    SWAP(b[1], b[0])


quarterround([0,1,2], [3,4,5]).display()

In [4]:
def toy_chacha():
    routine = QRoutine()
    n = 3
    indexes = [index for index in range(2*n)]
    a = indexes[:n]
    b = indexes[n:2*n]

    con = routine.new_wires(n)
    con_prime = routine.new_wires(n)
    iv0 = routine.new_wires(n)
    iv0_prime = routine.new_wires(n)
    iv1 = routine.new_wires(n)
    iv1_prime = routine.new_wires(n)
    key = routine.new_wires(n)
    key_prime = routine.new_wires(n)

    # Copy initial state
    for i in range(n):
        CNOT(con[i], con_prime[i])
        CNOT(iv0[i], iv0_prime[i])
        CNOT(iv1[i], iv1_prime[i])
        CNOT(key[i], key_prime[i])


    # Quarterrounds
    routine.apply(quarterround(a,b), con, iv0)
    routine.apply(quarterround(a,b), iv1, key)
    routine.apply(quarterround(a,b), key, con)
    routine.apply(quarterround(a,b), iv0, iv1)


    # Last Step
    routine.apply(add(a,b), con, con_prime)
    routine.apply(add(a,b), iv0, iv0_prime)
    routine.apply(add(a,b), iv1, iv1_prime)
    routine.apply(add(a,b), key, key_prime)

    return routine

rotina = toy_chacha()
rotina.display()

# Hands-on: Algoritmo de Grover

## Sumário do Algoritmo de Grover

Seja $N=2^n$. Consideramos um registrador de $n$ qubits de modo que um dos elementos da base computacional é dado por $|x_0\rangle$. Definimos inicialmente uma condição inicial uniforme

$$|\psi_0\rangle =  \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1} |x\rangle$$

Daí, definimos os operadores:

$$U_{f} |x\rangle|i\rangle=|x\rangle|i\oplus f(x)\rangle,$$
chamado de oráculo (codifica a função $f$ onde $f(x_0)=-1$ e $f(x)=0$ se $x\neq x_0$, onde $x_0$ é chamado elemento marcado).
O operador $G$ é
$$G = (2|\psi_0\rangle\langle \psi_0| - \mathrm{I}_N)\otimes \mathrm{I}_2, $$
chamado de operador de inversão em torno da média.

## Os passos do algoritmo

O algoritmo tem os seguintes passos (usando $n+1$ qubits)

1. Primeiro, preparar o estado inicial $|\psi_0\rangle|-\rangle$;

2. Aplicar o operador $U_f$;

3. Aplicar o operador $G$;

4. Repetir $t$ vezes os passos 2 e 3 onde

$$
t=\left\lfloor\frac{\pi}{4}\sqrt N\right\rfloor;
$$

5. Medir o primeiro registrador na base computacional.

A medição retorna o estado $x_0$ com probabilidade de aproximadamente $1 - \frac{1}{\sqrt{N}}$.

In [5]:
def oracle(n):
    # Toy ChaCha Enc
    routine = QRoutine()
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)
    
    con = routine.new_wires(n)
    con_prime = routine.new_wires(n)
    iv0 = routine.new_wires(n)
    iv0_prime = routine.new_wires(n)
    iv1 = routine.new_wires(n)
    iv1_prime = routine.new_wires(n)
    key = routine.new_wires(n)
    key_prime = routine.new_wires(n)

    wires = [con, con_prime, iv0, iv0_prime, iv1, iv1_prime, key, key_prime]

    
    encryption = toy_chacha()
    with routine.compute():
        routine.apply(encryption, wires)

    # Compare Key Stream
    X.ctrl(4*n)([con, iv0, iv1, key], ancilla)
    
    # Toy ChaCha Uncompute
    routine.uncompute()

    return routine

rotina = oracle(3)
rotina.display()

In [6]:
def diffusion(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in wires:
        H(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in wires:
        X(qubit)
    # Do multi-controlled-Z gate
    X.ctrl(len(wires))(wires, ancilla)
    # Apply transformation |11..1> -> |00..0>
    for qubit in wires:
        X(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in wires:
        H(qubit)

    return routine

rotina = diffusion(24)
rotina.display()

In [7]:
def initial_state(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)
    for q in wires:
        H(q)
    X(ancilla)
    H(ancilla)

    return routine

rotina = initial_state(24)
rotina.display()

In [8]:
def clean_ancilla(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)

    H(ancilla)
    X(ancilla)

    return routine

rotina = clean_ancilla(24)
rotina.display()

In [13]:
qprog = Program()
n = 3
num_reg = 4
num_qubits = num_reg*n
# --------------------------
# Initial state preparation
# --------------------------
qbits = qprog.qalloc(2*num_qubits)
cbits = qprog.calloc(num_qubits)
# Prepare initial state
#X(con[0])
qprog.apply(X,0)
#X(iv0[1])
qprog.apply(X,7)
#X(iv1[2])
qprog.apply(X,14)
#X(key[0])
qprog.apply(X,18)
#X(key[1])
qprog.apply(X,19)
#X(key[2])
qprog.apply(X,20)

#initial = initial_state(2*num_qubits)
oracle_operator = oracle(n)
#diffusion_operator = diffusion(2*num_qubits)
#clean = clean_ancilla(2*num_qubits)
#check = check_result(num_qubits)

# -------------------------
# Applying Grover Algorithm
# -------------------------
#initial(qbits)
#for step in range(int((np.pi/4)*np.sqrt(2**num_qubits))):
oracle_operator(qbits)
#diffusion_operator(qbits)
#clean(qbits)
#check(qbits)

In [14]:
circuit = qprog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
# Create a job
job = circuit.to_job(nbshots=1024, qubits=[[0,1,2], [6,7,8], [12,13,14], [18,19,20]])
qpu = PyLinalg()

# Submit the job to the QPU
result = qpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
     print("State %s: probability %s +/- %s amplitude %s " % (sample.state, sample.probability, sample.err, sample.amplitude))